# Esame di Metodi Numerici - 6 Luglio 2023
## Turno II -- Ore 11.45

## Esercizio 1


Nel file ``Test_II.mat`` sono memorizzate le matrici A1, A2 ed A3 ed i vettori b1,b2,b3. Risolvere i 3 sistemi lineari aventi ciascuno di essi come matrice dei coefficienti A1 e termine noto b1, A2 e termine noto b2, A3 e terimine noto b3  utilizzando il metodo più adatto per ciascuno di essi e  commentare i risultati ottenuti  giustificandoli alla luce della teoria.
Verificare se le matrici sono malcondizionate, dire teoricamente cosa questo implica e verificarlo sperimentalmente.


Per la lettura dei dati procedere nel seguente modo:

``from scipy.io import loadmat``

``import numpy as np``

``dati = loadmat('Test_II.mat')``

``A1=dati["A1"] ``

``A1=A1.astype(float)``

`` b1=dati["b1"] ``

`` b1=b1.astype(float)``

``A2=dati["A2"] ``

``A2=A2.astype(float)``

`` b2=dati["b2"] ``

`` b2=b2.astype(float)``

``A3=dati["A3"] ``

``A3=A3.astype(float)``

`` b3=dati["b3"] ``

`` b3=b3.astype(float)``



                                                                    Punti 16

In [6]:
from scipy.io import loadmat

import numpy as np

dati = loadmat('Test_II.mat')

A1=dati["A1"] 

A1=A1.astype(float)

b1=dati["b1"] 

b1=b1.astype(float)

A2=dati["A2"] 

A2=A2.astype(float)

b2=dati["b2"] 

b2=b2.astype(float)

A3=dati["A3"] 

A3=A3.astype(float)

b3=dati["b3"] 

b3=b3.astype(float)

In [13]:
def getInfo(A):
    print(A.shape)
    cond(A)
    checkSparsa(A)
    if A.shape[0] == A.shape[1]:
        simmetrica(A)
    maxRank(A)
    diagStrettamenteDominante(A)
    return

def simmetrica(A):
    if np.all(A.T==A):
        print("simmetrica")
        defPos(A)
        
def defPos(A):
    val,vec = np.linalg.eig(A)
    flag = val > 0
    if np.all(flag):
        print("def pos")
        
def maxRank(A):
    if np.linalg.matrix_rank(A) == min(A.shape):
        print("rango massimo")

def diagStrettamenteDominante(A):
    if np.all(np.diag(A)>np.abs(A-np.diag(A))):
        print("diagonale strettamente dominante")
        
def cond(A):
    print("indice di condizionamento => {:e}".format(np.linalg.cond(A)))

def checkSparsa(A):
    nonZero = np.count_nonzero(A)
    perc = nonZero/(A.shape[0]*A.shape[1]) * 100
    if perc <= 33:
        print("sparsa")
    else:
        print("densa")
    print(perc,"%")


In [15]:
getInfo(A1)
print("--------------------------")
getInfo(A2)
print("--------------------------")
getInfo(A3)
print("--------------------------")

(15, 15)
indice di condizionamento => 1.680111e+07
densa
100.0 %
simmetrica
def pos
rango massimo
diagonale strettamente dominante
--------------------------
(15, 6)
indice di condizionamento => 1.000000e+01
densa
100.0 %
rango massimo
--------------------------
(225, 225)
indice di condizionamento => 1.185268e+00
sparsa
2.528395061728395 %
rango massimo
--------------------------


A1 => dato che è mal condizionata, quadrata e densa potrei utilizzare il metodo QR, oppure dato che è simmetrica e definita positiva potrei utilizzare la fattorizzazione di cholensky.

A2 => sistema sovradeterminanto e ben condizionata quindi utilizzo il metodo delle equazioni normali.

A3 => questa matrice è di grandi dimensioni e sparsa, il problema è che non è ne simmetrica e definita positiva e ne a diagonale strettamente dominante quindi metodi come jacobi, steepestDescent e conjugate gradiente non sono indicati; rimane Gauss-Seidel e Gauss-SeidelSor anche se fanno potrebbero fare comunque un pò fatica a trovare la soluzione.

In [100]:
import RisolviSis as solver
import scipy.linalg as splin
def eqnorm(A,b):
#Risolve un sistema sovradeterminato con il metodo delle equazioni normali
    G= np.dot(A.T,A)
     
    f= np.dot(A.T,b)
    
    L= splin.cholesky(G,lower=True)
    U=L.T
        
   
    z,flag=solver.Lsolve(L,f)
    
    x,flag=solver.Usolve(U,z)
    
    return x

def cholesky(A,b):
    L = splin.cholesky(A,lower=True)
    
    y,flag = solver.Lsolve(L,b)
    
    x,flag = solver.Usolve(L.T,y)
    return x

def qr(A,b):
    Q,R = splin.qr(A)
    y = Q.T@b
    
    x,flag = solver.Usolve(R,y)
    return x

def gauss_seidel(A,b,x0,toll,it_max):
    errore=1000
    d=np.diag(A)
    D=np.diag(d)
    E=np.tril(A,-1)
    F=np.triu(A,1)
    M=(D+E)
    N=-F
    T=np.dot(np.linalg.inv(M),N)
    autovalori=np.linalg.eigvals(T)
    raggiospettrale=np.max(np.abs(autovalori))
    print("raggio spettrale Gauss-Seidel ",raggiospettrale)
    it=0
    er_vet=[]
    while it<it_max and errore >toll:
        temp=b-F@x0
        x,flag= solver.Lsolve(M,temp)
        errore=np.linalg.norm(x-x0)/np.linalg.norm(x)
        er_vet.append(errore)
        x0=x.copy()
        it=it+1
    return x,it,er_vet

In [106]:
xQR = qr(A1,b1)
print("metodo QR => ",xQR)
xC = cholesky(A1,b1)
print("metodo cholesky => ",xC)
print("-------FINE A1----------")
x2 = eqnorm(A2,b2)
print(x2)
print("-------FINE A2----------")
toll = 1e-9
itMax = 400
x0 = np.zeros((A3.shape[0],1))
xG,itG,erG = gauss_seidel(A3,b3,x0,toll,itMax)
print(xG)

metodo QR =>  [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
metodo cholesky =>  [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
-------FINE A1----------
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
-------FINE A2----------
raggio spettrale Gauss-Seidel  0.024750078276736077
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1

A1 è molto mal condizionata questo implica che un piccola perturbazione può portare ad avere dei risultati completamente distanti da quelli che ci si aspetta.
Mentre le altre due matrici sono ben condizionate quindi sicuramente se le perturbiamo allo stesso modo di A1 vedremo che i risultati ottenuti convergeranno molto di più verso la soluzione che ci si aspetta.
Infatti i risultati sotto sono una prova pratica della teoria appena esposta

In [112]:
A1_per = A1.copy()
A1_per[0][0] = A1_per[0][0]+ 0.01*A1_per[0][0]
A2_per = A2.copy()
A2_per[0][0] = A2_per[0][0]+ 0.01*A2_per[0][0]
A3_per = A3.copy()
A3_per[0][0] = A3_per[0][0]+ 0.01*A3_per[0][0]


xQR = qr(A1_per,b1)
print("metodo QR => ",xQR)
xC = cholesky(A1_per,b1)
print("metodo cholesky => ",xC)
print("-------FINE A1----------")
x2 = eqnorm(A2_per,b2)
print(x2)
print("-------FINE A2----------")
toll = 1e-9
itMax = 400
x0 = np.zeros((A3_per.shape[0],1))
xG,itG,erG = gauss_seidel(A3_per,b3,x0,toll,itMax)
print(xG)

metodo QR =>  [[ 9.18866321e-04]
 [ 1.70236887e+00]
 [ 1.46522171e+00]
 [ 1.22249656e+00]
 [ 8.14115407e-01]
 [ 2.07484245e+00]
 [ 3.07187444e+00]
 [-1.25718390e+00]
 [-4.71416794e-01]
 [ 5.69302343e-01]
 [ 9.57378407e-01]
 [ 2.16843904e+00]
 [ 1.48154298e+00]
 [ 9.75922442e-01]
 [ 5.84665025e-01]]
metodo cholesky =>  [[ 9.18866322e-04]
 [ 1.70236887e+00]
 [ 1.46522171e+00]
 [ 1.22249656e+00]
 [ 8.14115407e-01]
 [ 2.07484245e+00]
 [ 3.07187444e+00]
 [-1.25718390e+00]
 [-4.71416794e-01]
 [ 5.69302343e-01]
 [ 9.57378407e-01]
 [ 2.16843904e+00]
 [ 1.48154298e+00]
 [ 9.75922442e-01]
 [ 5.84665025e-01]]
-------FINE A1----------
[[0.99687043]
 [1.00204914]
 [1.0014001 ]
 [1.00027047]
 [1.00089772]
 [1.00076746]]
-------FINE A2----------
raggio spettrale Gauss-Seidel  0.02475007634386074
[[0.99009227]
 [0.99981633]
 [0.99999659]
 [0.99999994]
 [1.00000006]
 [1.00000341]
 [1.00000019]
 [1.00000001]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        

# Esercizio 2

Data l'equazione di 2° grado $$\frac{1}{2} x^2+2bx-c=0$$
che diventa $$x^2+4bx-2c=0$$

con $b=10^7, c=10^{-i},i=-5,\cdots,12 $

- a) Scrivere le due formule algebriche per ricavare i valori delle 2 soluzioni

                                                                                    1 punto
- b) Studiare l'indice di condizionamento delle 2 formule algebriche, facendo uso del risultato teorico visto a lezione riguardo l'indice di condizionamento della valutazione di una funzione. (Spiegare il significato di problema mal condizionato) e dire quale delle due formule è malcondizionata. Per quali valori di $c$ il problema risulta ben condizionato?

                                                                                    5 punti

- c) Nel caso in cui una delle due formule risulti mal condizionata, proporre una soluzione algebricamente equivalente che non sia malcondizionata.

                                                                                    3 punti

Totale 9 punti

In [172]:
delta =lambda a,b,c: np.sqrt(b**2-4*a*c)
x1 = lambda a,b,c: (-b+delta(a,b,c))/(2*a)
x2 = lambda a,b,c: (-b-delta(a,b,c))/(2*a)
i = np.arange(-5,13)
givenB = 10**7
givenC = np.array([])
for j in np.nditer(i):
    if j >=0:
        givenC = np.append(givenC,1/10**j)
    else:
        givenC = np.append(givenC,10**abs(j))

a = 1
b = 4*givenB
c = -2*givenC


print(x1(a,b,c))
print(x2(a,b,c))

[4.99999896e-03 5.00001013e-04 5.00008464e-05 4.99933958e-06
 4.99188900e-07 4.84287739e-08 3.72529030e-09 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[-40000000.005      -40000000.0005     -40000000.00005
 -40000000.000005   -40000000.0000005  -40000000.00000004
 -40000000.         -40000000.         -40000000.
 -40000000.         -40000000.         -40000000.
 -40000000.         -40000000.         -40000000.
 -40000000.         -40000000.         -40000000.        ]


In [174]:
f = lambda x:x**2+4*b*x-2*c    

fp = lambda x: 2*x+4*b 

k1Calc = (x1(a,b,c)**2+8*a*b*x1(a,b,c))/(x1(a,b,c)**2+8*a*b*x1(a,b,c)-8*a*c)
k2Calc = (x2(a,b,c)**2+8*a*b*x2(a,b,c))/(x2(a,b,c)**2+8*a*b*x2(a,b,c)-8*a*c)
print(k1Calc)
print(k2Calc)

[ 0.49999995  0.50000051  0.50000423  0.49996698  0.49959412  0.49201846
  0.42695316  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.         -0.        ]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


la prima formula è malcondizionata come possiamo notare, da quando i = 2 in poi i risultati non sono più attendibili per via dell'elevato spacing di b**2 come mostrato sotto:

In [181]:
print(4*a*c)
print("da questo valore in poi la sottrazione -4ac non viene più considerata =>",np.spacing(b**2))

[-8.00000000e+05 -8.00000000e+04 -8.00000000e+03 -8.00000000e+02
 -8.00000000e+01 -8.00000000e+00 -8.00000000e-01 -8.00000000e-02
 -8.00000000e-03 -8.00000000e-04 -8.00000000e-05 -8.00000000e-06
 -8.00000000e-07 -8.00000000e-08 -8.00000000e-09 -5.67349568e-09
 -6.58028836e-09  1.09983782e-08]
da questo valore in poi la sottrazione -4ac non viene più considerata => 0.25


## Domanda intelligenza artificiale

 Limiti delle reti MLP (multilayer Perceptron) ed introduzione delle Reti neurali Convoluzionali. 
- Architettura di una rete neurale convoluzionale: strati convoluzionali, attivazione non lineare e pooling + parte fully connected.
- Loss function per il task della regressione.  Training di una rete. 
- Cenni sull'algoritmo di backpropagation per il calcolo delle derivate parziale della funzione costo rispetto ai pesi ditutti i layer .
- Tecniche di Ottimizzazione: metodo di discesa del gradient batch, metodo del gradiente stocastico (SGD) ,metodo del gradiente stocastico minibatch.
                                                                                [7  punti]
